In [ ]:
%pip install --upgrade --quiet tiktoken langchain langgraph beautifulsoup4 langchain-community

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install -qU "langchain[openai]"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# Get the openai secret key:
import getpass

secret_key = getpass.getpass('Please enter your openai key:')

In [1]:
import os

os.environ["LANGSMITH_TRACING"] = "true"

In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

In [11]:
from langchain.chat_models import init_chat_model

chat_model = init_chat_model(model_provider="openai", model="gpt-4o-mini", api_key=secret_key)

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate

# Define prompt
prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\\n\\n{context}")]
)

# Instantiate chain
chain = create_stuff_documents_chain(chat_model, prompt)

# Invoke chain
result = chain.invoke({"context": docs})
print(result)

The article "LLM Powered Autonomous Agents" by Lilian Weng explores the concept and development of autonomous agents leveraging large language models (LLMs) as their core controller. It outlines an agent system framework consisting of three main components: 

1. **Planning**: This involves task decomposition, allowing agents to break down complex tasks into manageable subgoals, and self-reflection mechanisms to improve future actions based on past performance.

2. **Memory**: It categorizes memory into short-term and long-term types, highlighting the importance of memory in enabling agents to retain and retrieve information efficiently to enhance their decision-making capabilities.

3. **Tool Use**: It discusses various case studies where LLMs are integrated with external tools for improved functionality, such as in scientific discovery agents, and generative agents that simulate human behavior.

The article also examines the challenges faced in building these LLM-powered agents, inclu

In [13]:
#Stream tokens
for token in chain.stream({"context": docs}):
    print(token, end="|")

|The| article| "|LL|M| Powered| Autonomous| Agents|"| by| Lil|ian| W|eng| discusses| the| development| and| potential| of| autonomous| agents| powered| by| large| language| models| (|LL|Ms|).| It| provides| an| overview| of| the| system| architecture| that| integrates| planning|,| memory|,| and| tool| use| as| core| components|.

|1|.| **|Agent| System| Overview|**|:| L|LM|s| serve| as| the| brain| of| autonomous| agents|.| Key| components| include|:
|  | -| **|Planning|**|:| In|vol|ves| task| decomposition| and| self|-ref|lection| to| manage| complex| tasks| effectively| and| learn| from| past| actions|.
|  | -| **|Memory|**|:| Differ|enti|ates| between| short|-term| memory| (|in|-context| learning|)| and| long|-term| memory| (|external| information| storage|)| to| sustain| infinite| recall| capabilities| through| vector| stores| and| fast| retrieval| algorithms|.
|  | -| **|Tool| Use|**|:| Enh|ances| agent| capabilities| by| allowing| them| to| utilize| external| APIs| and| tools| to

In [16]:
# Also available via the hub: `hub.pull("rlm/reduce-prompt")`
reduce_template = """
The following is a set of summaries:
{docs}
Take these and distill it into a final, consolidated summary
of the main themes.
"""

reduce_prompt = ChatPromptTemplate([("human", reduce_template)])
print(reduce_prompt)

input_variables=['docs'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['docs'], input_types={}, partial_variables={}, template='\nThe following is a set of summaries:\n{docs}\nTake these and distill it into a final, consolidated summary\nof the main themes.\n'), additional_kwargs={})]


In [17]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)
print(f"Generated {len(split_docs)} documents.")

Created a chunk of size 1003, which is longer than the specified 1000


Generated 14 documents.


In [27]:
from langchain_core.prompts import ChatPromptTemplate

map_prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\\n\\n{context}")]
)

print(map_prompt)

input_variables=['context'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Write a concise summary of the following:\\n\\n{context}'), additional_kwargs={})]


In [ ]:
import operator
from typing import Annotated, List, Literal, TypedDict

from langchain.chains.combine_documents.reduce import (
    acollapse_docs,
    split_list_of_docs,
)
from langchain_core.documents import Document
from langgraph.constants import Send
from langgraph.graph import END, START, StateGraph

token_max = 1000


def length_function(documents: List[Document]) -> int:
    """Get number of tokens for input contents."""
    return sum(chat_model.get_num_tokens(doc.page_content) for doc in documents)


# This will be the overall state of the main graph.
# It will contain the input document contents, corresponding
# summaries, and a final summary.
class OverallState(TypedDict):
    # Notice here we use the operator.add
    # This is because we want combine all the summaries we generate
    # from individual nodes back into one list - this is essentially
    # the "reduce" part
    contents: List[str]
    summaries: Annotated[list, operator.add]
    collapsed_summaries: List[Document]
    final_summary: str


# This will be the state of the node that we will "map" all
# documents to in order to generate summaries
class SummaryState(TypedDict):
    content: str


# Here we generate a summary, given a document
async def generate_summary(state: SummaryState):
    #generate a promt to the llm model
    prompt = map_prompt.invoke(state["content"])
    response = await chat_model.ainvoke(prompt)
    # We return a dictionary with a key "summaries" and a list of summaries
    return {"summaries": [response.content]}


# Here we define the logic to map out over the documents
# We will use this an edge in the graph
def map_summaries(state: OverallState):
    # We will return a list of `Send` objects
    # Each `Send` object consists of the name of a node in the graph
    # as well as the state to send to that node
    return [
        # send is a special function that will send the data to the next node
        Send("generate_summary", {"content": content}) for content in state["contents"]
    ]


def collect_summaries(state: OverallState):
    return {
        "collapsed_summaries": [Document(summary) for summary in state["summaries"]]
    }


async def _reduce(input: dict) -> str:
    prompt = reduce_prompt.invoke(input)
    response = await chat_model.ainvoke(prompt)
    return response.content


# Add node to collapse summaries
async def collapse_summaries(state: OverallState):
    doc_lists = split_list_of_docs(
        state["collapsed_summaries"], length_function, token_max
    )
    results = []
    for doc_list in doc_lists:
        results.append(await acollapse_docs(doc_list, _reduce))

    return {"collapsed_summaries": results}


# This represents a conditional edge in the graph that determines
# if we should collapse the summaries or not
def should_collapse(
    state: OverallState,
) -> Literal["collapse_summaries", "generate_final_summary"]:
    num_tokens = length_function(state["collapsed_summaries"])
    if num_tokens > token_max:
        return "collapse_summaries"
    else:
        return "generate_final_summary"


# Here we will generate the final summary
async def generate_final_summary(state: OverallState):
    response = await _reduce(state["collapsed_summaries"])
    return {"final_summary": response}


# Construct the graph
# Nodes:
graph = StateGraph(OverallState)
graph.add_node("generate_summary", generate_summary)  # same as before
graph.add_node("collect_summaries", collect_summaries)
graph.add_node("collapse_summaries", collapse_summaries)
graph.add_node("generate_final_summary", generate_final_summary)

# Edges:
graph.add_conditional_edges(START, map_summaries, ["generate_summary"])
graph.add_edge("generate_summary", "collect_summaries")
graph.add_conditional_edges("collect_summaries", should_collapse)
graph.add_conditional_edges("collapse_summaries", should_collapse)
graph.add_edge("generate_final_summary", END)

app = graph.compile()

In [34]:
async for step in app.astream(
    {"contents": [doc.page_content for doc in split_docs]},
    {"recursion_limit": 10},
):
    print(list(step.keys()))

['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['collect_summaries']
['collapse_summaries']
['collapse_summaries']
['generate_final_summary']


In [35]:
print(step)

{'generate_final_summary': {'final_summary': 'The consolidated summary of the main themes from the two documents highlights key aspects of LLM-powered autonomous agents and software architecture implementation:\n\n1. **Core Components of LLM-Powered Agents**: These agents are built around critical elements such as Planning, Memory, and Tool Use. Effective planning techniques like Chain of Thought and Tree of Thoughts are essential for breaking down complex tasks, alongside mechanisms for self-reflection and iterative learning to improve performance.\n\n2. **Addressing Challenges in LLMs**: The importance of managing hallucinations is emphasized, with techniques like Chain of Hindsight and Algorithm Distillation enhancing output quality and supporting policy development through reinforcement learning. Additionally, limitations in LLMs related to self-reflection and long-term planning are noted, particularly in adapting to historical information and errors.\n\n3. **Memory and Learning Pr

In [41]:
print(step["generate_final_summary"])

{'final_summary': 'The consolidated summary of the main themes from the two documents highlights key aspects of LLM-powered autonomous agents and software architecture implementation:\n\n1. **Core Components of LLM-Powered Agents**: These agents are built around critical elements such as Planning, Memory, and Tool Use. Effective planning techniques like Chain of Thought and Tree of Thoughts are essential for breaking down complex tasks, alongside mechanisms for self-reflection and iterative learning to improve performance.\n\n2. **Addressing Challenges in LLMs**: The importance of managing hallucinations is emphasized, with techniques like Chain of Hindsight and Algorithm Distillation enhancing output quality and supporting policy development through reinforcement learning. Additionally, limitations in LLMs related to self-reflection and long-term planning are noted, particularly in adapting to historical information and errors.\n\n3. **Memory and Learning Processes**: The discussion i